# Export monthly mean AHF at two sites
- This script is used to export monthly mean AHF from simulations and AH4GUC dataset;
- Simulations: CNTL, TRAF at FR-Capitole and UK-Manchester. 

In [4]:
import rasterio
from rasterio.windows import Window
import pandas as pd
import matplotlib.pyplot as plt
import string
from itertools import product
import xarray as xr
home_path = '/gws/nopw/j04/duicv/yuansun/'
# ref: https://figshare.com/articles/dataset/Global_1-km_present_and_future_hourly_anthropogenic_heat_flux/12612458/6

In [17]:
lat_list = [43.6045, 53.480821]
lon_list = [1.4454, -2.232823]
site_names = ['FR-Capitole', 'UK-Manchester']
monthly_data = {site: [] for site in site_names}

for month in range(1, 13):
    monthly_file = f'{home_path}dataset/AH4GUC/version6/AHE_2010_{month:02d}_average.tif'
    with rasterio.open(monthly_file) as src:
        for l, site in enumerate(site_names):
            lat_flux, lon_flux = lat_list[l], lon_list[l]
            row, col = src.index(lon_flux, lat_flux)
            window = Window(col, row, 1, 1)
            value = src.read(1, window=window)[0, 0] / 100000  # Adjusting unit
            monthly_data[site].append(value)

# Convert to DataFrame
df_monthly = pd.DataFrame(monthly_data, index=[f'{i:02d}' for i in range(1, 13)]).T
df_monthly.index.name = 'Site'
df_monthly.columns.name = 'Month'
df_monthly


Month,01,02,03,04,05,06,07,08,09,10,11,12
Site,,,,,,,,,,,,
FR-Capitole,48.10851,48.22127,44.45487,42.32866,41.41099,36.99039,36.10105,35.45168,36.42510,38.69040,45.77249,47.45086
UK-Manchester,23.91677,24.29349,24.57351,22.43931,20.98508,19.49859,17.53218,18.27031,19.67391,20.22477,22.84021,22.60166


In [18]:
values_fr = df_monthly.loc['FR-Capitole'].values
values_uk = df_monthly.loc['UK-Manchester'].values

In [19]:
df_fr = pd.read_csv(f'{home_path}0_urban_traffic/output_analysis/capitole/AHF/data_for_figure/FR-Capitole.csv')
df_fr['traffic_contribution'] = 100*df_fr['TRAFFICFLUX'] / df_fr['AHF']
df_fr.head()

,time,month,hour,year,minute,diurnal,URBAN_AC,URBAN_HEAT,VENTILATION,WASTEHEAT,FGR_U,TRAFFICFLUX,AHF,case,traffic_contribution
0,2004-02-20 01:00:00,2,1,2004,0,01:00,0.0,21.110060,5.942730,4.155546,-90.72937,0.0,25.265606,cntl,0.0
1,2004-02-20 01:30:00,2,1,2004,30,01:30,0.0,21.637337,5.791717,4.222012,-79.33860,0.0,25.859348,cntl,0.0
2,2004-02-20 02:00:00,2,2,2004,0,02:00,0.0,22.147747,5.939824,4.327467,-70.29194,0.0,26.475214,cntl,0.0
3,2004-02-20 02:30:00,2,2,2004,30,02:30,0.0,22.663204,6.076488,4.429549,-68.21493,0.0,27.092754,cntl,0.0
4,2004-02-20 03:00:00,2,3,2004,0,03:00,0.0,23.162970,6.222219,4.532641,-68.52138,0.0,27.695610,cntl,0.0


In [20]:
df_uk = pd.read_csv(f'{home_path}0_urban_traffic/output_analysis/manchester/AHF/data_for_figure/UK-Manchester.csv')
df_uk['traffic_contribution'] = 100*df_uk['TRAFFICFLUX_traffic'] / df_uk['AHF_traffic']
df_uk['time'] = pd.to_datetime(df_uk['time']) - pd.DateOffset(hours=1)
df_uk['month'] = df_uk['time'].dt.month
df_uk.head()

,time,FSH_U_cntl,URBAN_AC_cntl,URBAN_HEAT_cntl,VENTILATION_cntl,WASTEHEAT_cntl,FGR_U_cntl,FSH_U_traffic,URBAN_AC_traffic,URBAN_HEAT_traffic,...,FGR_U_traffic,TRAFFICFLUX_traffic,TRAFFICFLUX_cntl,AHF_cntl,AHF_traffic,hour,minute,diurnal,traffic_contribution,month
0,2022-01-01 00:00:00,-26.368145,0.0,6.343987,1.505959,1.300586,4.241446,-24.313444,0.0,6.343987,...,7.016270,7.949450,0,7.644573,15.594023,1,0,01:00,50.977544,1
1,2022-01-01 01:00:00,-32.981693,0.0,6.237513,1.256034,1.268797,2.243642,-31.504507,0.0,6.218744,...,5.486924,7.945894,0,7.506310,15.433435,2,0,02:00,51.484936,1
2,2022-01-01 02:00:00,-44.191975,0.0,6.184282,1.107646,1.247503,-6.258418,-43.318310,0.0,6.163151,...,-5.295407,3.972919,0,7.431785,11.379818,3,0,03:00,34.911970,1
3,2022-01-01 03:00:00,-44.220050,0.0,6.317245,1.057381,1.236856,-14.563049,-42.835136,0.0,6.302743,...,-14.162316,3.972851,0,7.554101,11.508225,4,0,04:00,34.521840,1
4,2022-01-01 04:00:00,-43.612620,0.0,6.435579,1.330521,1.263449,-16.814860,-42.210540,0.0,6.421233,...,-16.283443,3.972911,0,7.699028,11.654694,5,0,05:00,34.088509,1


In [21]:
months = range(1, 13)
sites = ['FR-Capitole', 'UK-Manchester']
cases = ['AH4GUC', 'cntl', 'traffic']
variables = ['AHF', 'traffic_contribution']
case_list = ['cntl', 'traffic']
# Create all combinations
combinations = list(product(months, cases, sites, variables))

# Create DataFrame
df_combined = pd.DataFrame(combinations, columns=['month', 'case', 'site',  'variable'])
df_combined['value'] = None
df_combined.loc[
    (df_combined['case'] == 'AH4GUC') & (df_combined['site'] == 'FR-Capitole') & (df_combined['variable'] == 'AHF'),
    'value'
] = values_fr
df_combined.loc[
    (df_combined['case'] == 'AH4GUC') & (df_combined['site'] == 'UK-Manchester') & (df_combined['variable'] == 'AHF'),
    'value'
] = values_uk
for case in case_list:
    df_fr_case = df_fr[df_fr['case'] == case]
    for variable in variables:
        df_combined.loc[(df_combined['case'] == case) & (df_combined['site'] == 'FR-Capitole') & (df_combined['variable'] == variable),'value'] = df_fr_case.groupby('month')[variable].mean().values
for case in case_list:
    df_uk_case = df_uk[['month', f'AHF_{case}']].groupby('month').mean()
    df_combined.loc[(df_combined['case'] == case) & (df_combined['site'] == 'UK-Manchester') & (df_combined['variable'] == 'AHF'),'value'] = df_uk_case.values
df_combined.loc[(df_combined['case'] == 'traffic') & (df_combined['site'] == 'UK-Manchester') & (df_combined['variable'] == 'traffic_contribution'),'value'] = df_uk[['month', 'traffic_contribution']].groupby('month').mean().reset_index()['traffic_contribution'].values

df_combined.to_csv('data_for_figure/monthly_mean.csv', index=False)
df_combined

,month,case,site,variable,value
0,1,AH4GUC,FR-Capitole,AHF,48.10851
1,1,AH4GUC,FR-Capitole,traffic_contribution,None
2,1,AH4GUC,UK-Manchester,AHF,23.91677
3,1,AH4GUC,UK-Manchester,traffic_contribution,None
4,1,cntl,FR-Capitole,AHF,19.601595
...,...,...,...,...,...
139,12,cntl,UK-Manchester,traffic_contribution,None
140,12,traffic,FR-Capitole,AHF,35.324971
141,12,traffic,FR-Capitole,traffic_contribution,62.85191
142,12,traffic,UK-Manchester,AHF,40.170168


# AH-DMSP
- ref: https://doi.org/10.6084/m9.figshare.c.3647903.v1

In [8]:
ds_ah_dmsp = xr.open_dataset(f'{home_path}dataset/AH-DMSP/2010.nc')
ds_ah_dmsp_ahf = ds_ah_dmsp['AHF']
ds_ah_dmsp_ahf

<xarray.DataArray 'AHF' (lat: 16800, lon: 43200)> Size: 6GB
[725760000 values with dtype=float64]
Coordinates:
  * lon      (lon) float32 173kB -180.0 -180.0 -180.0 ... 179.6 179.6 179.6
  * lat      (lat) float32 67kB 75.0 75.0 74.99 74.98 ... -64.98 -64.99 -65.0
Attributes:
    units:          Wm-2
    long_name:      annual anthropogenic heat flux: 2010
    standard_name:  AHF
    grid_mapping:   crs

In [10]:
lat_list = [43.6045, 53.480821]
lon_list = [1.4454, -2.232823]
site_names = ['FR-Capitole', 'UK-Manchester']
results = []

for name, lat, lon in zip(site_names, lat_list, lon_list):
    value = ds_ah_dmsp_ahf.sel(lat=lat, lon=lon, method='nearest').values.item()
    results.append({'Site': name, 'Latitude': lat, 'Longitude': lon, 'AHF': value})

df = pd.DataFrame(results)
print(df)

            Site   Latitude  Longitude  AHF
0    FR-Capitole  43.604500   1.445400  0.1
1  UK-Manchester  53.480821  -2.232823  0.6


# AHF by Jin et al.
- https://doi.org/10.1038/s41597-019-0143-1

In [16]:
lat_list = [43.6045, 53.480821]
lon_list = [1.4454, -2.232823]
site_names = ['FR-Capitole', 'UK-Manchester']

file = f'{home_path}dataset/AHF_Jin/2015/global_AHF2015.tif'
with rasterio.open(file) as src:
      for l, site in enumerate(site_names):
        lat_flux, lon_flux = lat_list[l], lon_list[l]
        row, col = src.index(lon_flux, lat_flux)
        window = Window(col, row, 1, 1)
        value = src.read(1, window=window)[0, 0]
        print(value)

19.609047142822266
29.915361517578123
